In [106]:
######### loading the packages
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.dates as mdates
import matplotlib.patches as patches
import ipywidgets as widgets
import voila
from IPython.display import display, HTML, clear_output
import io

In [2]:
# Instead of a global directory, just use relative folders:
# os.chdir("/Users/ruofeiguo/CEO Compensation/my-timeline-repo")
data_directory = "data"
# graph_directory = "graph"

In [3]:
base = pd.read_csv(os.path.join(data_directory, 'SumComp.csv'))
base['CIK'] = base['CIK'].astype(str).str.replace(r"[^\d]", "", regex=True).astype(int)
# # base.to_csv(os.path.join(data_directory, 'SumComp.csv'), index=False)

In [66]:
grant = pd.read_csv(os.path.join(data_directory, 'GpbaGrant_clean.csv'))
# grant_award = pd.read_csv(os.path.join(data_directory, 'GpbaGrant_CEO.csv'))
# # print(grant_award.head)
# grant = grant.merge(grant_award[['grantID', 'AwardType']], on='grantID', how='left')
# # print(grant.head)
# grant.to_csv(os.path.join(data_directory, 'GpbaGrant_clean.csv'), index=False)
# print(grant.head)

In [5]:
time = pd.read_csv(os.path.join(data_directory, 'GpbaTime_vest.csv'))

In [48]:
df_abs = pd.read_csv(os.path.join(data_directory, 'GpbaAbs_vest_clean.csv'), encoding="latin1")
df_rel = pd.read_csv(os.path.join(data_directory, 'GpbaRel_vest.csv'))
columns_to_display = [
        'CIK', 'participantid', 'companyName', 'metric', 'fiscalYear', 
        'grantDate', 'startDate', 'endDate', 'size', 'percentVest', 'value', 'grantId', 'group'
    ]
df_abs_filtered = df_abs[columns_to_display]
df_rel_filtered = df_rel[columns_to_display]
df_comp = pd.concat([df_abs_filtered, df_rel_filtered], ignore_index=True)

df_comp['participantid'] = df_comp['participantid'].astype(int)

# unique_list = (
#     df_comp[['CIK', 'companyName', 'participantid']]
#     .drop_duplicates()
#     .values
#     .tolist()
# )

unique_cik = (
    df_comp[['CIK', 'companyName']]
    .drop_duplicates()
    .values
    .tolist()
)
cik_to_company = {cik: name for cik, name in unique_cik}
company_to_cik = {name: cik for cik, name in unique_cik}

unique_ceo = (
    df_comp[['companyName', 'participantid', 'fiscalYear']]
    .drop_duplicates()
)
unique_ceo = unique_ceo.loc[unique_ceo.groupby(['companyName', 'participantid'])['fiscalYear'].idxmin()]
company_to_ceo = (
    unique_ceo.sort_values(by=['companyName', 'fiscalYear'])
    .groupby('companyName')['participantid']
    .apply(list)
    .to_dict()
)

# Function to get participantid list for a given company
def get_participants_for_company(company_val):
    return company_to_ceo.get(company_val, [])  # Returns empty list if not found

all_company_names = (
    df_comp['companyName']
    .drop_duplicates()
    .values
    .tolist()
)

In [36]:
df_part = pd.read_csv(os.path.join(data_directory, 'ParticipantFY.csv'), encoding="latin1", low_memory=False)
participant_dict = df_part.drop_duplicates(subset=['participantid', 'fullName']).set_index('participantid')['fullName'].to_dict()

# df_part = df_part[['participantid', 'fullName', 'FiscalYear']]
# print(df_part.head)
# df_part = df_part.dropna(subset=['participantid'])
# df_part = df_part.dropna(subset=['FiscalYear'])
# df_part.to_csv(os.path.join(data_directory, 'ParticipantFY.csv'), index=False)

# def resolve_ceoname_id_conflicts(df_comp):
#     # Step 1: Resolve multiple companyNames for the same CIK
#     df_comp_sorted = df_comp.sort_values(by=['participantid', 'FiscalYear', 'fullName'], ascending=[True, False, False])

#     # Select the best companyName for each CIK (latest fiscalYear, longest name)
#     best_company_mapping = df_comp_sorted.drop_duplicates(subset=['participantid'], keep='first').set_index('participantid')['fullName']

#     # Map the best companyName to all rows with that CIK
#     df_comp['fullName'] = df_comp['participantid'].map(best_company_mapping)

#     # Step 2: Resolve multiple CIKs for the same companyName
#     cik_counts = df_comp.groupby('fullName')['participantid'].transform('nunique')  # Unique CIKs per company
#     multiple_cik_mask = cik_counts > 1  # Identify companies linked to multiple CIKs

#     # Assign ranks to CIKs within the same company
#     df_comp['cik_rank'] = df_comp.groupby('fullName')['participantid'].rank(method='dense', ascending=True).astype("Int64")

#     # Rename only when needed
#     df_comp.loc[multiple_cik_mask, 'fullName'] = df_comp['fullName'] + "(" + df_comp['cik_rank'].astype(str) + ")"

#     # Drop the helper column
#     df_comp.drop(columns=['cik_rank'], inplace=True)

#     # Step 3: Verify One-to-One Mapping
#     cik_to_name_counts = df_comp.groupby('participantid')['fullName'].nunique()
#     name_to_cik_counts = df_comp.groupby('fullName')['participantid'].nunique()

#     assert cik_to_name_counts.max() == 1, "Some IDs still map to multiple CEOs!"
#     assert name_to_cik_counts.max() == 1, "Some CEOs still map to multiple IDs!"
    
#     # Ensure the shape remains the same
#     assert df_comp.shape[0] == original_shape, "Row count changed! No rows should be dropped."

#     return df_comp

# Apply the function
# original_shape = df_part.shape[0]
# df_final = resolve_ceoname_id_conflicts(df_part)

# # Display the final DataFrame
# print(df_final.head)
# df_final.to_csv(os.path.join(data_directory, 'ParticipantFY.csv'), index=False)

In [8]:
# df_rel = pd.read_csv(os.path.join(data_directory, 'GpbaRel_vest.csv'))
# df_rel['companyName'] = df_rel['CIK'].map(cik_to_company)
# df_rel.to_csv(os.path.join(data_directory, 'GpbaRel_vest.csv'), index=False)

In [9]:
def create_df_combined(company_name=None, participantid=None, dis=False):
    """
    Reads the CSV files, filters on company_name (substring match, case-insensitive),
    CIK, and participantid if they are provided. Returns the combined DataFrame with
    columns: [CIK, companyName, metric, fiscalYear, grantDate, startDate, endDate, value, absRel].
    """

    # --- Read CSVs ---
    df_abs = pd.read_csv(os.path.join(data_directory, 'GpbaAbs_vest_clean.csv'))
    df_rel = pd.read_csv(os.path.join(data_directory, 'GpbaRel_vest.csv'))

    #### get CIK from unique list
    CIK = company_to_cik[company_name]

    # --- Filter the DataFrames ---
    df_abs_filtered = df_abs[(df_abs['CIK'] == CIK) & (df_abs['participantid'] == float(participantid))]
    df_rel_filtered = df_rel[(df_rel['CIK'] == CIK) & (df_rel['participantid'] == float(participantid))]

    if df_abs_filtered.empty and df_rel_filtered.empty:
        print(f"No data found for CIK={CIK}, participant={participantid}.")
        return  # nothing else to plot
    
    # --- Select columns and label abs/rel ---
    columns_to_display = [
        'CIK', 'companyName', 'metric', 'fiscalYear', 
        'grantDate', 'startDate', 'endDate', 'size', 'percentVest', 'value', 'grantId', 'group'
    ]
    df_abs_filtered = df_abs_filtered[columns_to_display].copy()
    df_rel_filtered = df_rel_filtered[columns_to_display].copy()

    df_abs_filtered['absRel'] = 'abs'
    df_rel_filtered['absRel'] = 'rel'

    df_combined = pd.concat([df_abs_filtered, df_rel_filtered], ignore_index=True)
    df_combined = df_combined.sort_values(by=['metric', 'grantDate'], ascending=[True, True])

    mask = (
        df_combined['percentVest'].isna() &
        df_combined['size'].isna() &
        (df_combined['metric'] == 'Unknown') &
        (df_combined['startDate'] == df_combined['grantDate']) &
        (df_combined['grantDate'] == df_combined['endDate'])
    )
    df_combined = df_combined[~mask]


    # Display the combined DataFrame (in notebook)
    if dis:
        pd.set_option('display.max_rows', None)
        pd.set_option('display.max_columns', None)
        display(df_combined)

    return df_combined

In [127]:
def create_intro_widget(content, is_button=False, grant_id=None):
    """
    Returns a widget (Button or Label) instead of updating VBox immediately.
    """
    if is_button and grant_id is not None:
        button = widgets.Button(
            description=content,
            layout=widgets.Layout(width="auto"),  # Auto width
            tooltip=f"Click to view details for Grant {grant_id}"
        )
        def on_button_click(b):
            with output_area:
                clear_output(wait=True)
                print(f"Grant {grant_id} details:")
        button.on_click(on_button_click)
        return button  # Return the button instead of appending

    else:
        return widgets.Label(value=content)  # Return a plain label

In [130]:
def plot_compensation_timeline_by_grant(company_name, participantid, dis=False):
    award_color_map = {
        'Option': 'dimgray', 'reloadOption': 'dimgray', 'phantomOption': 'dimgray',  # Dark grey
        'stock': 'lightgray', 'rsu': 'lightgray', 'sarEquity': 'lightgray', 
        'sarCash': 'lightgray', 'phantomStock': 'lightgray', 'Performance Unit': 'lightgray',  # Light grey
    }
    award_label_map = {
        'Option': 'Option Award', 'reloadOption': 'Option Award', 'phantomOption': 'Option Award',  # Dark grey
        'stock': 'Stock Award', 'rsu': 'Stock Award', 'sarEquity': 'Stock Award', 
        'sarCash': 'Stock Award', 'phantomStock': 'Stock Award', 'Performance Unit': 'Stock Award',  # Light grey
    }
    global grant_intro_box
    intro_widgets = []
    grant_text = widgets.HTML(
        f"<b>Information on metrics and their groupings:</b>"
    )    
    intro_widgets.append(grant_text)

    df_combined = create_df_combined(company_name=company_name, participantid=participantid, dis=dis)
    if df_combined.empty:
        print(f"No data found for company={company_name}, CIK={CIK}, participant={participantid}.")
        return

    # Just in case, ensure these are datetimes
    df_combined['grantDate'] = pd.to_datetime(df_combined['grantDate'], errors='coerce')
    df_combined['startDate'] = pd.to_datetime(df_combined['startDate'], errors='coerce')
    df_combined['endDate']   = pd.to_datetime(df_combined['endDate'], errors='coerce')

    CIK = company_to_cik[company_name]
    
    # --- Retrieve base salary data ---
    if pd.isna(CIK):  # If company_name is used, map it to CIK
        CIK = df_combined['CIK'].iloc[0] if not df_combined.empty else None

    base_filtered = base[(base['CIK'] == CIK) & (base['participantid'] == int(participantid))]
    time_filtered = time[(time['CIK'] == CIK) & (time['participantid'] == int(participantid))]

    # Convert fiscalYear to integer for sorting
    base_filtered.loc[:,'FiscalYear'] = pd.to_numeric(base_filtered['FiscalYear'], errors='coerce')
    
    # --- Compute the minimum value for scaling line thickness ---
    max_value = pd.concat([df_combined['size'], base_filtered['salary'], base_filtered['stockAwards'], base_filtered['optionAwards'], time_filtered['grantDateFV']], axis=0).dropna().max()

    # Group by grantId
    grouped = df_combined.groupby('grantId', dropna=False)
    group_grant_dates = {grantId: sub_df['grantDate'].min() for grantId, sub_df in grouped}
    sorted_groups = sorted(group_grant_dates.items(), key=lambda x: x[1])

    # Prepare the figure
    fig_width = 12  # Keep a fixed width
    fig_height = max(8, (len(grouped)+1) * 0.5)  # Scale height based on number of items
    fig, ax = plt.subplots(figsize=(fig_width, fig_height))

    # For coloring, reuse your color scheme if you want
    # palette = sns.color_palette("colorblind", 10)
    color_map = {'abs': '#4C72B0', 'rel': '#F4A261'}
    
    # We'll keep track of the y-position for each group
    current_y = 10
    y_positions = []
    y_labels = []

    # Track legend entries
    legend_labels = {
        'absolute, performance period': False,
        'relative, performance period': False,
        'Grant Date': False,
        'Non-performance-based': False,
        'Time-based': False,
        'Stock Award': False,
        'Cash Award': False,
        'Option Award': False
    }

    # --- Plot Base Salary ---
    max_salary_award = 0 if base_filtered['salary'].dropna().empty else max(20 * base_filtered['salary'].dropna().max() / max_value, 0.3)
    max_stock_award = 0 if base_filtered['stockAwards'].dropna().empty else max(20 * base_filtered['stockAwards'].dropna().max() / max_value, 0.3)
    max_option_award = 0 if base_filtered['optionAwards'].dropna().empty else max(20 * base_filtered['optionAwards'].dropna().max() / max_value, 0.3)
    base_salary_y = current_y
    stock_y = base_salary_y + max_salary_award/2 + 4 + max_stock_award/2
    option_y = stock_y + max_stock_award/2 + 4 + max_option_award/2
    ax.axhline(y=base_salary_y + max_salary_award/2 + 2, color='gray', linestyle='dashed', alpha=0.3)
    ax.axhline(y=stock_y + max_stock_award/2 + 2, color='gray', linestyle='dashed', alpha=0.3)
    for _, row in base_filtered.iterrows():
        fiscal_year = int(row['FiscalYear'])
        salary_value = row['salary']
        salary_height = 0
        if pd.notna(salary_value):
            height = max(20 * salary_value / max_value, 1)
            # Create a rectangle for base salary
            base_salary_rect = patches.Rectangle(
                (pd.Timestamp(f"{fiscal_year}-01-01"), base_salary_y - height / 2),  # (x, y) position
                pd.Timedelta(days=365),  # Width covering the full year
                height,  # Height scaled to salary value
                facecolor='black',
                edgecolor='black', 
                label='Non-performance-based'
            )
            
            ax.add_patch(base_salary_rect)  # Add rectangle to the plot

            legend_labels['Non-performance-based'] = True

        stock_value = row['stockAwards']
        if pd.notna(stock_value):
            height = max(20 * stock_value / max_value, 1)
            # Create a rectangle for base salary
            stock_rect = patches.Rectangle(
                (pd.Timestamp(f"{fiscal_year}-01-01"), stock_y - height / 2),  # (x, y) position
                pd.Timedelta(days=365),  # Width covering the full year
                height,  # Height scaled to salary value
                facecolor='black',
                edgecolor='black', 
                label='Non-performance-based'
            )
            
            ax.add_patch(stock_rect)  # Add rectangle to the plot
            legend_labels['Non-performance-based'] = True

        option_value = row['optionAwards']
        if pd.notna(option_value):
            height = max(20 * option_value / max_value, 1)
            # Create a rectangle for base salary
            option_rect = patches.Rectangle(
                (pd.Timestamp(f"{fiscal_year}-01-01"), option_y - height / 2),  # (x, y) position
                pd.Timedelta(days=365),  # Width covering the full year
                height,  # Height scaled to salary value
                facecolor='black',
                edgecolor='black', 
                label='Non-performance-based'
            )
            
            ax.add_patch(option_rect)  # Add rectangle to the plot
            legend_labels['Non-performance-based'] = True

    current_y = option_y + max_option_award/2 + 2
    # --- Plot Time-Based ---
    ax.axhline(y=current_y, color='gray', linestyle='dashed', alpha=0.3)
    # print(time_filtered)
    time_filtered = time_filtered.sort_values(by='grantDate')
    time_y = current_y
    current_y += 2
    all_nan = time_filtered['grantDateFV'].isna().all()
    
    for _, row in time_filtered.iterrows():
        grantDateFV = row['grantDateFV']
        sDate = pd.to_datetime(row['startDate'])
        eDate = pd.to_datetime(row['endDate'])
        gDate = pd.to_datetime(row['grantDate'])
        oops = False
        if sDate == eDate:
            eDate += pd.Timedelta(days=10)
            oops = True
        
        if pd.notna(grantDateFV):
            height = max(20 * grantDateFV / max_value, 1)
            # Create a rectangle for base salary
            rect = patches.Rectangle(
                (sDate, current_y),  # (x, y) position
                eDate - sDate,  # width
                height,  # height in data units
                color='#E9C46A',
                label='Time-based'
            )
            
            ax.add_patch(rect)  # Add rectangle to the plot
            if oops == True:
                ax.scatter(gDate, current_y+height/2, color='black', marker='o', s=6, label="Grant Date" if not legend_labels['Grant Date'] else "", zorder=0)
            else:
                ax.scatter(gDate, current_y+height/2, color='black', marker='o', s=6, label="Grant Date" if not legend_labels['Grant Date'] else "", zorder=5)
            legend_labels['Time-based'] = True
            legend_labels['Grant Date'] = True

            current_y += height+2
        else:
            ax.hlines(
                y=current_y,
                xmin=sDate,
                xmax=eDate,
                color="#E9C46A",
                linestyle="dashed",
                linewidth=1.5,  # Adjust thickness of dashed line
                label='Time-based' if all_nan else ""
            )
            if oops == True:
                ax.scatter(gDate, current_y, color='black', marker='o', s=6, label="Grant Date" if not legend_labels['Grant Date'] else "", zorder=0)
            else:
                ax.scatter(gDate, current_y, color='black', marker='o', s=6, label="Grant Date" if not legend_labels['Grant Date'] else "", zorder=5)
            legend_labels['Time-based'] = True
            legend_labels['Grant Date'] = True
            current_y += 2

    current_y += 2
    time_y = (current_y + time_y)/2
        

    # --- Plot Performance-Based ---
    sorted_grant_ids = sorted(grouped.groups.keys(), key=lambda x: group_grant_dates[x])

    for grant_id in sorted_grant_ids:
        low = current_y
        performance_grouping = grant.loc[grant['grantID'] == grant_id, 'performanceGrouping']
    
        # If grant_id is found, get the first matching value
        performance_grouping = performance_grouping.iloc[0] if not performance_grouping.empty else None
        
        ax.axhline(y=current_y, color='gray', linestyle='dashed', alpha=0.3)
        # old_y = current_y
        group_df = grouped.get_group(grant_id)  # Retrieve the actual DataFrame
        label_for_group = (f"grant {grant_id}" if pd.notna(grant_id) else "unknown grant") 
        group_df = group_df.copy()
        group_df['metric_absrel'] = group_df['metric']+group_df['absRel']

        novalue = False
        if group_df['value'].isna().all():
            novalue = True
        else:
            first_value = group_df['value'].iloc[0] if not group_df['value'].isna().all() else min_value
            total_height = max(20 * (first_value / max_value) if max_value else 20,1)
            # 1) Check if percentVest has NaNs
            if group_df['percentVest'].isna().any():
                use_count_based = True
                total_count = len(group_df)
                group_df['computedVest'] = total_height / total_count  # Equal height for all rows
            else:
                use_count_based = False
                total_percentVest = group_df['percentVest'].sum()
                if total_percentVest == 0 or pd.isna(total_percentVest):
                    total_percentVest = 1  # Avoid division by zero
                group_df['computedVest'] = np.maximum(group_df['percentVest'] / total_percentVest * total_height, 0.8)
        
        # Identify distinct combos within this group
        # Step 1: Create a unique schedule for each metric
        # Convert schedule into a tuple set per metric
        schedule_dict = {}
        for metric, group in group_df.groupby('metric_absrel'):
            unique_schedule = group[['grantDate', 'startDate', 'endDate']].drop_duplicates()
            schedule = tuple(unique_schedule.apply(tuple, axis=1))
            schedule_dict[metric] = schedule
        
        # Step 2: Group metrics by identical schedules
        schedule_to_metrics = {}
        for metric, schedule in schedule_dict.items():
            if schedule in schedule_to_metrics:
                schedule_to_metrics[schedule].append(metric)
            else:
                schedule_to_metrics[schedule] = [metric]
        
        # Step 3: Assign group labels to metrics
        metric_to_group = {}
        for group_idx, (schedule, metrics) in enumerate(schedule_to_metrics.items(), start=1):
            for metric in metrics:
                metric_to_group[metric] = f"Group_{group_idx}"
        
        # Step 4: Add group information back to the original DataFrame
        group_df = group_df.copy()
        group_df['metric_group'] = group_df['metric_absrel'].map(metric_to_group)
        # print(group_df)

        further_groups = group_df.groupby(['metric_group'], dropna=False)
        
        # If only 1 distinct timeline, we label the y-axis with all metrics from the entire group
        # else, we label each timeline line with its metrics
        if len(further_groups) == 1:
            
            # print(grant_id,1)
            # There's just one combo => we can directly get that
            metric_group, metric_df = list(further_groups)[0]
            combo_groups = metric_df.groupby(['grantDate', 'startDate', 'endDate'], dropna=False)

            sorted_metrics = list(set(metric_df['metric'].tolist()[::-1]))
            metric_label = ", ".join(sorted_metrics) if sorted_metrics else "No Metric"

            grant_text = widgets.HTML(
                f"<b>Grant ID:</b> {grant_id} <br>"
                f"<b>Grouping:</b> {performance_grouping} <br>"
                f"<b>Metrics:</b>"
            )
            intro_widgets.append(grant_text)

            metric_buttons = [
                widgets.Button(description=metric, layout=widgets.Layout(width="auto"))
                for metric in sorted_metrics
            ]

            for btn in metric_buttons:
                btn.on_click(on_metric_button_click)
            # Step 3: Arrange text + buttons horizontally
            button_box = widgets.HBox(metric_buttons)  # Display buttons in one row
            intro_widgets.append(button_box)  # Add button box to the list


            current_y += 6
            base_y = current_y
            
            for (gDate, sDate, eDate), sub_df in combo_groups:
                # Plot one line
                if novalue:
                    # print(grant_id,'novalue')
                    color = color_map.get(row['absRel'], 'gray')
                    ax.hlines(
                        y=current_y,
                        xmin=sDate,
                        xmax=eDate,
                        color=color,
                        linestyle="dashed",
                        linewidth=1.5  # Adjust thickness of dashed line
                    )
                    ax.scatter(gDate, current_y, color='black', marker='o', s=20, label="Grant Date" if not legend_labels['Grant Date'] else "", zorder=5)
                    legend_labels['Grant Date'] = True
                else:
                    sub_df = sub_df.sort_values(by=['absRel', 'computedVest'], ascending=[False, True])
                    # 3) Compute the vertical stacking based on computedVest
                    start_y = current_y
                    for _, row in sub_df.iterrows():
                        height = row['computedVest']
                        color = color_map.get(row['absRel'], 'gray')
        
                        line_label = 'absolute, performance period' if row['absRel'] == 'abs' else 'relative, performance period'

                        # print(grant_id,metric,sDate,eDate,start_y)
                        # Create a rectangle patch instead of using `ax.plot`
                        rect = patches.Rectangle(
                            (sDate, start_y),  # (x, y) position
                            eDate - sDate,  # width
                            height,  # height in data units
                            facecolor=color,
                            edgecolor=None,  # Optional: add border for better visibility
                            linewidth=0.2,  # Keep a thin border
                            label=line_label if not legend_labels[line_label] else None
                        )
                        
                        ax.add_patch(rect)  # Add rectangle to the plot
        
                        # Draw a dashed line between stacked rectangles (except for the last one)
                        if _ != sub_df.index[-1]:  
                            ax.hlines(
                                y=start_y + height,  # Position at top of current rectangle
                                xmin=sDate,
                                xmax=eDate,
                                color="white",
                                linestyle="dashed",
                                linewidth=0.5  # Adjust thickness of dashed line
                            )
        
                        
                        legend_labels[line_label] = True  # Mark legend entry as used
                
                        # Move up to stack the next segment
                        start_y += height  # Ensure consistent stacking in data units
                
                    # Plot the grant date marker
                    ax.scatter(gDate, (current_y + start_y) / 2, color='black', marker='o', s=6, label="Grant Date" if not legend_labels['Grant Date'] else "", zorder=5)
                    legend_labels['Grant Date'] = True
                    current_y = start_y+3
    
            # We'll label the entire group's y-tick with the list of metrics
            y_positions.append((base_y+ current_y)/2)
            y_labels.append(f"{label_for_group}")

            current_y += 6
        
        else:
            # print(grant_id,0)
            # Multiple distinct combos => each timeline gets its own line
            # We'll label each line with the relevant metrics
            lines_in_group = 0
            current_y += 6
            count = 0
            grant_text = widgets.HTML(
                f"<b>Grant ID:</b> {grant_id} <br>"
                f"<b>Grouping:</b> {performance_grouping} <br>"
            )
            intro_widgets.append(grant_text)
            
            for metric_group, metric_df in further_groups:
                base_y = current_y
                count += 1
                if count >1:
                    current_y += 6
                    ax.axhline(y=current_y, color='gray', linestyle='dashed', alpha=0.3)
                    current_y += 6
                base_y = current_y
                    
                combo_groups = metric_df.groupby(['grantDate', 'startDate', 'endDate'], dropna=False)

                sorted_metrics = list(set(metric_df['metric'].tolist()[::-1]))
                
                metric_label = ", ".join(sorted_metrics) if sorted_metrics else "No Metric"
    
                grant_text = widgets.HTML(
                    f"<b>Metrics:</b>"
                )
                intro_widgets.append(grant_text)
                metric_buttons = [
                    widgets.Button(description=metric, layout=widgets.Layout(width="auto"))
                    for metric in sorted_metrics
                ]
                for btn in metric_buttons:
                    btn.on_click(on_metric_button_click)
        
                # Step 3: Arrange text + buttons horizontally
                button_box = widgets.HBox(metric_buttons)  # Display buttons in one row
                intro_widgets.append(button_box)  # Add button box to the list

                for (gDate, sDate, eDate), sub_df in combo_groups:
                    # Plot one line
                    if novalue:
                        color = color_map.get(row['absRel'], 'gray')
                        ax.hlines(
                            y=current_y,
                            xmin=sDate,
                            xmax=eDate,
                            color=color,
                            linestyle="dashed",
                            linewidth=1.5  # Adjust thickness of dashed line
                        )
                        ax.scatter(gDate, current_y, color='black', marker='o', s=20, label="Grant Date" if not legend_labels['Grant Date'] else "", zorder=5)
                        legend_labels['Grant Date'] = True
                    else:
                        sub_df = sub_df.sort_values(by=['absRel', 'computedVest'], ascending=[False, True])
                        # 3) Compute the vertical stacking based on computedVest
                        start_y = current_y
                        for _, row in sub_df.iterrows():
                            height = row['computedVest']
                            color = color_map.get(row['absRel'], 'gray')
            
                            line_label = 'absolute, performance period' if row['absRel'] == 'abs' else 'relative, performance period'
            
                            # Create a rectangle patch instead of using `ax.plot'
                            rect = patches.Rectangle(
                                (sDate, start_y),  # (x, y) position
                                eDate - sDate,  # width
                                height,  # height in data units
                                facecolor=color,
                                edgecolor=None,  # Optional: add border for better visibility
                                linewidth=0.2,  # Keep a thin border
                                label=line_label if not legend_labels[line_label] else None
                            )
                            
                            ax.add_patch(rect)  # Add rectangle to the plot
            
                            # Draw a dashed line between stacked rectangles (except for the last one)
                            if _ != sub_df.index[-1]:
                                ax.hlines(
                                    y=start_y + height,  # Position at top of current rectangle
                                    xmin=sDate,
                                    xmax=eDate,
                                    color="white",
                                    linestyle="dashed",
                                    linewidth=0.5  # Adjust thickness of dashed line
                                )
            
                            
                            legend_labels[line_label] = True  # Mark legend entry as used
                    
                            # Move up to stack the next segment
                            start_y += height  # Ensure consistent stacking in data units
                    
                        # Plot the grant date marker
                        ax.scatter(gDate, (current_y + start_y) / 2, color='black', marker='o', s=6, label="Grant Date" if not legend_labels['Grant Date'] else "", zorder=5)
                        legend_labels['Grant Date'] = True
                        current_y = start_y+3
                        
                # We'll label the entire group's y-tick with the list of metrics
                y_positions.append((base_y+ current_y)/2)
                y_labels.append(f"{label_for_group}({count})")
    
            current_y += 6

        award = grant.loc[grant['grantID'] == grant_id, 'AwardType'].iloc[0] if not grant.loc[grant['grantID'] == grant_id, 'AwardType'].empty else None
        color = award_color_map.get(award, 'white')
        label = award_label_map.get(award, 'Cash Award')
        ax.axhspan(low, current_y, color=color, alpha=1, zorder=-1, label=label)
        legend_labels[label] = True
        

    # Decorate axes
    ax.set_xlabel("Time")
    ax.set_ylabel("Grant-level grouping")

    # Format x-axis as Year-Month
    ax.xaxis.set_major_locator(mdates.YearLocator())
    ax.xaxis.set_minor_locator(mdates.MonthLocator())
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
    plt.xticks(rotation=45)

    # y-ticks
    ax.set_yticks(y_positions + [time_y, option_y, stock_y, base_salary_y])
    ax.set_yticklabels(y_labels + ["Time-Based Awards", "Option Awards", "Stock Awards", "Base Salary"])

    ax.grid(axis='x', linestyle='--', alpha=0.6)

    # --- Ensure Proper Legend ---
    handles, labels = ax.get_legend_handles_labels()
    desired_order = ['absolute, performance period', 'relative, performance period', 'Grant Date', 'Non-performance-based', 'Time-based', 'Stock Award','Cash Award','Option Award']
    final_labels = [lbl for lbl in desired_order if legend_labels.get(lbl, False)]
    # Replace the handle for "Cash Award" with a custom one (with a black border)
    custom_handles = []
    for lbl in final_labels:
        if lbl == 'Cash Award':  # Override existing "Cash Award" entry
            custom_handles.append(patches.Patch(facecolor='white', edgecolor='black', label='Cash Award'))
        else:
            custom_handles.append(handles[labels.index(lbl)])
    # ax.legend(
    #     ordered_handles, final_labels, 
    #     loc='upper center',  # Places legend at the top center
    #     bbox_to_anchor=(0.5, 1.15),  # Moves it above the plot
    #     ncol=len(final_labels),  # Spread items in a single row
    #     frameon=False  # Optional: Remove legend border
    # )
    # Create the legend above the graph without shrinking it
    fig.subplots_adjust(top=0.8)  # Increase top margin without shrinking the plot
    
    # Add legend as a separate figure element
    fig.legend(
        custom_handles, final_labels,
        loc='upper center',  # Place the legend at the top center
        bbox_to_anchor=(0.5, 1.05),  # Position it above the graph
        ncol=len(final_labels),  # Arrange items in one row
        frameon=False  # Remove legend border
    )

    plt.tight_layout()
    plt.title(f"Grant-Level Timeline for {df_combined['companyName'].iloc[0]}, CEO {participant_dict[participantid]}")
    plt.show()
    grant_intro_box.children = tuple(intro_widgets)

In [132]:
# Create an Output widget to capture logs
error_output = widgets.Output()
output_area = widgets.Output()
button_container = widgets.HBox()

# Placeholder dictionary to store generated plots for each participant
graph_store = {}

def filter_company_names(substring):
    """Return a list of company names containing 'substring' (case-insensitive)."""
    s_lower = substring.lower()
    return [n for n in all_company_names if s_lower in n.lower()]

def on_company_typed(change):
    if change['name'] == 'value':
        typed = change['new'].strip()  # Remove extra whitespace
        # Filter company suggestions
        company_widget.options = [n for n in all_company_names if typed.lower() in n.lower()]

        # Enable button only if the exact company name is in all_company_names
        run_button.disabled = typed not in all_company_names

        # If typed is an exact known name => narrow CIK options
        if typed in all_company_names:
            matched_ciks = ciks_for_company(typed)
            CIK_widget.options = matched_ciks

            # If the current typed CIK isn't in matched_ciks, clear the CIK
            if CIK_widget.value not in matched_ciks:
                CIK_widget.value = ""
        else:
            CIK_widget.options = []
            CIK_widget.value = ""
            
def on_run_button_click(b):
    """
    Clears the output area, then runs the plotting function.
    """
    with error_output:
        error_output.clear_output()  # Clear previous messages
        
        company_val = company_widget.value

        if company_val not in all_company_names:
            print("Invalid company name")  # Display error message
            return  # Stop execution

        with output_area:
            output_area.clear_output()
        
        # Now call your timeline function(s)
        participant_list = get_participants_for_company(company_val)

        # Clear previous buttons
        button_container.children = []
    
        # Create buttons for each CEO
        buttons = [
            widgets.Button(description=f"{participant_dict[participant]}", layout=widgets.Layout(width="200px", height="auto", overflow="visible"))
            for participant in participant_list
        ]

        # Attach event handlers
        for btn, participant in zip(buttons, participant_list):
            btn.on_click(lambda b, p=participant: plot_selected_graph(company_val, p))
    
        # Display buttons
        button_container.children = buttons

def plot_selected_graph(company_val, participant_val):
    """
    Clears the output area and plots the graph for the selected CEO.
    """
    with output_area:
        output_area.clear_output(wait=True)
        plot_compensation_timeline_by_grant(company_val, participant_val, False)

def on_metric_button_click(b):
    with metric_output:  # Direct output to the output widget
        metric_output.clear_output()
        print(f"The pay-performance relationship for metric: {b.description}")

# --- Create dynamic Comboboxes for Company & CIK ---
company_widget = widgets.Combobox(
    placeholder='Type a company name...',
    options=all_company_names,
    description='Company:'
)
company_widget.value = "Citigroup INC"  # your existing default
company_widget.observe(on_company_typed, names='value')

run_button = widgets.Button(description="Generate Timeline")
output_area = widgets.Output()

# Create a title widget
title_widget = widgets.HTML(
    value="<h1 style='text-align: center;'>Visualizing CEO Compensation</h1>"
)

run_button.on_click(on_run_button_click)

# Create the Clear button
clear_button = widgets.Button(description="Clear", button_style='warning')

# Function to clear input fields
def on_clear_button_click(b):
    company_widget.value = ""

# Attach event to Clear button
clear_button.on_click(on_clear_button_click)
grant_intro_box = widgets.VBox([])

metric_output = widgets.Output()

# Display the widgets & output in the notebook
display(
    widgets.VBox([
        title_widget,  # Add the title at the top
        widgets.HBox([company_widget]),
        widgets.HBox([run_button, clear_button]),
        error_output,
        button_container,
        output_area,
        metric_output,
        grant_intro_box
    ])
)